# Geography of the Amazon IN 
Sale Report
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geopandas as gpd
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file into a Pandas DataFrame
zipcode_df = pd.read_csv("Amazon_Sale_Report.csv")
coordinates_df = pd.read_csv("India_States-UTs.csv")

# Display sample data
zipcode_df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [12]:
zipcode_df.drop(['Fulfilment', 'ship-country', 'promotion-ids', 'B2B', 'fulfilled-by', 'Unnamed: 22', 'Style', 'SKU', 'Size', 'ASIN' ],axis=1, inplace=True)
zipcode_df.head()

,index,Order ID,Date,Status,Sales Channel,ship-service-level,Category,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code
0,0,405-8078784-5731545,04-30-22,Cancelled,Amazon.in,Standard,Set,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Amazon.in,Standard,kurta,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon.in,Expedited,kurta,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0
3,3,403-9615377-8133951,04-30-22,Cancelled,Amazon.in,Standard,Western Dress,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon.in,Expedited,Top,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0


In [13]:
zipcode_df_cleaned = zipcode_df.dropna(subset=['ship-postal-code'])
zipcode_df_cleaned['ship-postal-code'] = zipcode_df_cleaned['ship-postal-code'].astype(int)
zipcode_df_cleaned.head()


,index,Order ID,Date,Status,Sales Channel,ship-service-level,Category,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code
0,0,405-8078784-5731545,04-30-22,Cancelled,Amazon.in,Standard,Set,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Amazon.in,Standard,kurta,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon.in,Expedited,kurta,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210
3,3,403-9615377-8133951,04-30-22,Cancelled,Amazon.in,Standard,Western Dress,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon.in,Expedited,Top,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073


--- 

### Bring in the API of India's zip code information

India has a total of 28 States and 8 Union Territories with at least 720 districts.<br>
The Indian postal departmenthas allotted a unique postal code of pin code to each<br>
district/village/town/city to ensure quick delivery of postal services.

In [14]:
# API base URL
url = "https://api.postalpincode.in/pincode/"
state_data = []

# Counters
record_count = 1
set_count = 1

print(zipcode_df_cleaned['ship-postal-code'][0]) #test
state_url = url + str(zipcode_df_cleaned['ship-postal-code'][0])
state_info = requests.get(state_url).json()
print(state_info[0]["PostOffice"][0]["State"]) #test

# Loop through all the zip codes to pull the state
for zipcode in zipcode_df_cleaned['ship-postal-code']:
    state_url = url + str(zipcode)
    try:
        state_info = requests.get(state_url).json()
        state_name = state_info[0]["PostOffice"][0]["State"]
        state_data.append(state_name)
    except: # need except for all try blocks
        print("Zip code not found. Skipping...")

zipcode_df_cleaned["State/UT"] = state_data
zipcode_df_cleaned.head()

400081
Maharashtra


,index,Order ID,Date,Status,Sales Channel,ship-service-level,Category,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,State/UT
0,0,405-8078784-5731545,04-30-22,Cancelled,Amazon.in,Standard,Set,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081,Maharashtra
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Amazon.in,Standard,kurta,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085,Karnataka
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon.in,Expedited,kurta,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210,Maharashtra
3,3,403-9615377-8133951,04-30-22,Cancelled,Amazon.in,Standard,Western Dress,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008,Pondicherry
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon.in,Expedited,Top,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073,Tamil Nadu


In [15]:
state_list = zipcode_df_cleaned['State/UT'].unique()
state_list

array(['Maharashtra', 'Karnataka', 'Pondicherry', 'Tamil Nadu',
       'Uttar Pradesh', 'Chandigarh', 'Telangana', 'Andhra Pradesh',
       'Rajasthan', 'Delhi', 'Haryana', 'Assam', 'Jharkhand',
       'Chattisgarh', 'Odisha'], dtype=object)

In [16]:
#Coordinates
coordinates_df.columns = [['State/UT', 'Lat', 'Lng']]
coordinates_df.head()

,State/UT,Lat,Lng
0,Andaman and Nicobar Islands,11.7401,92.6586
1,Andhra Pradesh,15.9129,79.7400
2,Arunachal Pradesh,28.2180,94.7278
3,Assam,31.1048,77.1734
4,Bihar,25.0961,85.3131


In [17]:
state_coord = pd.merge(zipcode_df_cleaned, coordinates_df, on='State/UT')

ValueError: The column label 'State/UT' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

---

### Trying to create a map that displays a point for every shipping postal code (destination).

In [ ]:
%%capture --no-display

# Configure the map plot
shipped_state = zipcode_df_cleaned["State"]
map_plot_1 = zipcode_df_cleaned.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    alpha=.75
)

# Display the map
map_plot_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

# Display sample data


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel


# Print a message to follow up the hotel search


# Iterate through the hotel_df DataFrame


# Display sample data


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot


# Display the map
